# 🎯 VAE, ELBO & Variational Inference

This notebook implements and visualizes:
1. ELBO derivation and intuition
2. Full VAE implementation in PyTorch
3. Training dynamics and loss curves
4. Latent space visualization
5. The effect of KL weight (β)
6. Information bottleneck perspective

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from typing import Tuple
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

## 1. ELBO: The Evidence Lower Bound

$$\mathcal{L}(\phi, \theta; x) = \underbrace{\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)]}_{\text{Reconstruction}} - \underbrace{D_{KL}(q_\phi(z|x) \| p(z))}_{\text{KL Regularization}}$$

In [ ]:
# Visualize the ELBO decomposition

def visualize_elbo_decomposition():
    """Show how ELBO relates to log p(x) and KL gap."""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: ELBO components
    ax = axes[0]
    
    # Simulate training progress
    epochs = np.arange(100)
    
    # Reconstruction improves
    recon = -100 * np.exp(-epochs/30) - 5
    
    # KL starts low (posterior = prior), increases, then stabilizes
    kl = 20 * (1 - np.exp(-epochs/20)) * (1 - 0.3*np.exp(-epochs/50))
    
    elbo = recon - kl
    
    ax.plot(epochs, -recon, 'b-', linewidth=2, label='Reconstruction Loss')
    ax.plot(epochs, kl, 'r-', linewidth=2, label='KL Divergence')
    ax.plot(epochs, -elbo, 'g--', linewidth=2, label='Total Loss (-ELBO)')
    
    ax.set_xlabel('Training Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('VAE Loss Components During Training')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Right: ELBO vs true log p(x)
    ax = axes[1]
    
    # True log p(x) is what we want but can't compute
    log_px = -50 + 45 * (1 - np.exp(-epochs/40))
    
    # ELBO is always below
    elbo_normalized = log_px - 5 * (1 + np.exp(-epochs/30))
    
    # Gap = KL(q || p_true_posterior)
    gap = log_px - elbo_normalized
    
    ax.fill_between(epochs, elbo_normalized, log_px, alpha=0.3, color='red', 
                    label='Gap = KL(q || p_posterior)')
    ax.plot(epochs, log_px, 'b-', linewidth=2, label='log p(x) (unknown)')
    ax.plot(epochs, elbo_normalized, 'g--', linewidth=2, label='ELBO (what we maximize)')
    
    ax.set_xlabel('Training Epoch')
    ax.set_ylabel('Value')
    ax.set_title('ELBO is a Lower Bound on log p(x)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_elbo_decomposition()

## 2. VAE Implementation in PyTorch

In [ ]:
class VAE(nn.Module):
    """
    Variational Autoencoder
    
    Architecture:
        Encoder: x -> h -> (mu, log_var)
        Reparameterization: z = mu + sigma * epsilon
        Decoder: z -> h -> x_reconstructed
    """
    
    def __init__(self, input_dim: int, hidden_dim: int, latent_dim: int):
        super().__init__()
        
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Latent space parameters
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_log_var = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()  # Output in [0, 1]
        )
    
    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Encode input to latent distribution parameters."""
        h = self.encoder(x)
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var
    
    def reparameterize(self, mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        """
        Reparameterization trick: z = mu + sigma * epsilon
        Allows gradients to flow through stochastic sampling.
        """
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        z = mu + std * epsilon
        return z
    
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """Decode latent code to reconstruction."""
        return self.decoder(z)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Full forward pass."""
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var


def vae_loss(x: torch.Tensor, x_recon: torch.Tensor, 
             mu: torch.Tensor, log_var: torch.Tensor,
             beta: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    VAE Loss = Reconstruction + beta * KL
    
    Reconstruction: MSE or BCE depending on data
    KL: Closed-form for Gaussian to standard normal
    
    KL = 0.5 * sum(mu^2 + sigma^2 - 1 - log(sigma^2))
    """
    # Reconstruction loss (per sample)
    recon_loss = F.mse_loss(x_recon, x, reduction='sum') / x.size(0)
    
    # KL divergence (per sample)
    # D_KL(N(mu, sigma^2) || N(0, 1)) = 0.5 * sum(mu^2 + sigma^2 - 1 - log(sigma^2))
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / x.size(0)
    
    # Total loss
    total_loss = recon_loss + beta * kl_loss
    
    return total_loss, recon_loss, kl_loss


print("VAE model defined!")
print("\nKey components:")
print("1. Encoder: x -> (μ, log σ²)")
print("2. Reparameterization: z = μ + σ ⊙ ε, ε ~ N(0,I)")
print("3. Decoder: z -> x̂")
print("4. Loss: MSE(x, x̂) + β · KL(q(z|x) || p(z))")

In [ ]:
# Visualize the KL loss components

def visualize_kl_components():
    """Show how each part of KL loss works."""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # KL loss for different mu (sigma=1)
    ax = axes[0]
    mu_range = np.linspace(-3, 3, 100)
    kl_mu = 0.5 * mu_range**2  # When log_var=0 (sigma=1)
    
    ax.plot(mu_range, kl_mu, 'b-', linewidth=2)
    ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
    ax.set_xlabel('μ')
    ax.set_ylabel('KL contribution')
    ax.set_title('Mean Penalty: $\\frac{1}{2}\\mu^2$\n"Pull mean toward 0"')
    ax.grid(True, alpha=0.3)
    
    # KL loss for different sigma (mu=0)
    ax = axes[1]
    log_var_range = np.linspace(-3, 2, 100)
    sigma_sq = np.exp(log_var_range)
    kl_sigma = 0.5 * (sigma_sq - 1 - log_var_range)  # When mu=0
    
    ax.plot(sigma_sq, kl_sigma, 'b-', linewidth=2)
    ax.axvline(x=1, color='r', linestyle='--', alpha=0.5)
    ax.set_xlabel('σ²')
    ax.set_ylabel('KL contribution')
    ax.set_title('Variance Penalty: $\\frac{1}{2}(\\sigma^2 - 1 - \\log\\sigma^2)$\n"Pull variance toward 1"')
    ax.set_xlim(0, 5)
    ax.grid(True, alpha=0.3)
    
    # Combined KL as heatmap
    ax = axes[2]
    mu_grid = np.linspace(-3, 3, 100)
    logvar_grid = np.linspace(-2, 2, 100)
    MU, LOGVAR = np.meshgrid(mu_grid, logvar_grid)
    
    # KL = 0.5 * (mu^2 + sigma^2 - 1 - log(sigma^2))
    KL = 0.5 * (MU**2 + np.exp(LOGVAR) - 1 - LOGVAR)
    
    contour = ax.contourf(MU, np.exp(LOGVAR/2), KL, levels=20, cmap='viridis')
    plt.colorbar(contour, ax=ax, label='KL [nats]')
    ax.plot(0, 1, 'r*', markersize=15, label='Minimum (μ=0, σ=1)')
    ax.set_xlabel('μ')
    ax.set_ylabel('σ')
    ax.set_title('Total KL Loss\nMinimum at standard normal')
    ax.legend()
    
    plt.tight_layout()
    plt.show()

visualize_kl_components()

## 3. Synthetic Data: 2D Points from Mixture of Gaussians

In [ ]:
def generate_2d_clusters(n_samples=2000, n_clusters=5):
    """Generate 2D data from mixture of Gaussians."""
    
    # Cluster centers arranged in a circle
    angles = np.linspace(0, 2*np.pi, n_clusters, endpoint=False)
    centers = 3 * np.stack([np.cos(angles), np.sin(angles)], axis=1)
    
    # Generate points
    samples_per_cluster = n_samples // n_clusters
    data = []
    labels = []
    
    for i, center in enumerate(centers):
        cluster_data = np.random.randn(samples_per_cluster, 2) * 0.5 + center
        data.append(cluster_data)
        labels.extend([i] * samples_per_cluster)
    
    data = np.concatenate(data, axis=0)
    labels = np.array(labels)
    
    # Normalize to [0, 1]
    data_min = data.min()
    data_max = data.max()
    data_normalized = (data - data_min) / (data_max - data_min)
    
    return data_normalized, labels, (data_min, data_max)

# Generate data
data, labels, (data_min, data_max) = generate_2d_clusters()

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(data[:, 0], data[:, 1], c=labels, cmap='tab10', alpha=0.5, s=10)
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Synthetic 2D Data: 5 Gaussian Clusters')
ax.set_aspect('equal')
plt.colorbar(scatter, label='Cluster')
plt.show()

print(f"Data shape: {data.shape}")
print(f"Data range: [{data.min():.3f}, {data.max():.3f}]")

## 4. Training the VAE

In [ ]:
def train_vae(data: np.ndarray, latent_dim: int = 2, hidden_dim: int = 64,
              n_epochs: int = 100, batch_size: int = 64, lr: float = 1e-3,
              beta: float = 1.0) -> Tuple[VAE, dict]:
    """
    Train VAE on data.
    
    Args:
        data: Training data (N, D)
        latent_dim: Dimension of latent space
        hidden_dim: Hidden layer dimension
        n_epochs: Number of training epochs
        batch_size: Batch size
        lr: Learning rate
        beta: KL weight (beta-VAE)
    
    Returns:
        Trained VAE model and training history
    """
    
    input_dim = data.shape[1]
    
    # Create model
    model = VAE(input_dim, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Create dataloader
    tensor_data = torch.FloatTensor(data).to(device)
    dataset = TensorDataset(tensor_data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Training history
    history = {'total_loss': [], 'recon_loss': [], 'kl_loss': []}
    
    # Training loop
    model.train()
    for epoch in range(n_epochs):
        epoch_total = 0
        epoch_recon = 0
        epoch_kl = 0
        n_batches = 0
        
        for batch in dataloader:
            x = batch[0]
            
            # Forward pass
            x_recon, mu, log_var = model(x)
            
            # Compute loss
            loss, recon, kl = vae_loss(x, x_recon, mu, log_var, beta)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_total += loss.item()
            epoch_recon += recon.item()
            epoch_kl += kl.item()
            n_batches += 1
        
        history['total_loss'].append(epoch_total / n_batches)
        history['recon_loss'].append(epoch_recon / n_batches)
        history['kl_loss'].append(epoch_kl / n_batches)
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{n_epochs} | "
                  f"Total: {history['total_loss'][-1]:.4f} | "
                  f"Recon: {history['recon_loss'][-1]:.4f} | "
                  f"KL: {history['kl_loss'][-1]:.4f}")
    
    return model, history

In [ ]:
# Train VAE
print("Training VAE with β=1.0...")
print("="*50)

model, history = train_vae(
    data, 
    latent_dim=2, 
    hidden_dim=64,
    n_epochs=100,
    batch_size=64,
    beta=1.0
)

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ax = axes[0]
ax.plot(history['total_loss'], 'b-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Total Loss (-ELBO)')
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(history['recon_loss'], 'g-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Reconstruction Loss')
ax.grid(True, alpha=0.3)

ax = axes[2]
ax.plot(history['kl_loss'], 'r-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('KL Divergence')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Visualizing the Latent Space

In [ ]:
def visualize_latent_space(model: VAE, data: np.ndarray, labels: np.ndarray):
    """Visualize encoder outputs in latent space."""
    
    model.eval()
    with torch.no_grad():
        tensor_data = torch.FloatTensor(data).to(device)
        mu, log_var = model.encode(tensor_data)
        z = model.reparameterize(mu, log_var)
        
        mu = mu.cpu().numpy()
        z = z.cpu().numpy()
        sigma = np.exp(0.5 * log_var.cpu().numpy())
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Latent means
    ax = axes[0]
    scatter = ax.scatter(mu[:, 0], mu[:, 1], c=labels, cmap='tab10', alpha=0.5, s=10)
    ax.set_xlabel('z₁')
    ax.set_ylabel('z₂')
    ax.set_title('Latent Space (μ only)\nClusters preserved!')
    ax.set_aspect('equal')
    plt.colorbar(scatter, ax=ax, label='Cluster')
    
    # Sampled z
    ax = axes[1]
    scatter = ax.scatter(z[:, 0], z[:, 1], c=labels, cmap='tab10', alpha=0.5, s=10)
    ax.set_xlabel('z₁')
    ax.set_ylabel('z₂')
    ax.set_title('Latent Space (sampled z)\nWith added noise')
    ax.set_aspect('equal')
    plt.colorbar(scatter, ax=ax, label='Cluster')
    
    # Variance distribution
    ax = axes[2]
    ax.hist(sigma.flatten(), bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(x=1.0, color='r', linestyle='--', linewidth=2, label='Target σ=1')
    ax.set_xlabel('σ')
    ax.set_ylabel('Count')
    ax.set_title('Encoder σ Distribution\nKL pulls toward 1')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    return mu, z, sigma

mu, z, sigma = visualize_latent_space(model, data, labels)

In [ ]:
# Visualize reconstruction
def visualize_reconstruction(model: VAE, data: np.ndarray, labels: np.ndarray, n_samples: int = 500):
    """Compare original data with reconstructions."""
    
    model.eval()
    idx = np.random.choice(len(data), n_samples, replace=False)
    
    with torch.no_grad():
        x = torch.FloatTensor(data[idx]).to(device)
        x_recon, _, _ = model(x)
        x_recon = x_recon.cpu().numpy()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    ax = axes[0]
    ax.scatter(data[idx, 0], data[idx, 1], c=labels[idx], cmap='tab10', alpha=0.5, s=20)
    ax.set_xlabel('x₁')
    ax.set_ylabel('x₂')
    ax.set_title('Original Data')
    ax.set_aspect('equal')
    
    ax = axes[1]
    ax.scatter(x_recon[:, 0], x_recon[:, 1], c=labels[idx], cmap='tab10', alpha=0.5, s=20)
    ax.set_xlabel('x₁')
    ax.set_ylabel('x₂')
    ax.set_title('Reconstructed Data')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    # Reconstruction error
    mse = np.mean((data[idx] - x_recon)**2)
    print(f"Mean Squared Error: {mse:.6f}")

visualize_reconstruction(model, data, labels)

## 6. Generation: Sampling from the Prior

In [ ]:
def generate_samples(model: VAE, n_samples: int = 500):
    """Generate new samples by sampling from prior."""
    
    model.eval()
    with torch.no_grad():
        # Sample from prior p(z) = N(0, I)
        z = torch.randn(n_samples, model.latent_dim).to(device)
        
        # Decode
        x_generated = model.decode(z)
        
        z = z.cpu().numpy()
        x_generated = x_generated.cpu().numpy()
    
    return z, x_generated

# Generate samples
z_prior, x_generated = generate_samples(model)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Prior samples
ax = axes[0]
ax.scatter(z_prior[:, 0], z_prior[:, 1], alpha=0.5, s=10, c='purple')
ax.set_xlabel('z₁')
ax.set_ylabel('z₂')
ax.set_title('Samples from Prior\nz ~ N(0, I)')
ax.set_aspect('equal')
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)

# Generated data
ax = axes[1]
ax.scatter(x_generated[:, 0], x_generated[:, 1], alpha=0.5, s=10, c='green')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Generated Data\nDecoder(z)')
ax.set_aspect('equal')

# Compare with real data
ax = axes[2]
ax.scatter(data[:500, 0], data[:500, 1], alpha=0.3, s=10, c='blue', label='Real')
ax.scatter(x_generated[:, 0], x_generated[:, 1], alpha=0.3, s=10, c='red', label='Generated')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Real vs Generated')
ax.set_aspect('equal')
ax.legend()

plt.tight_layout()
plt.show()

## 7. The Effect of β: Information Bottleneck

In [ ]:
# Train VAEs with different beta values
betas = [0.0, 0.5, 1.0, 2.0, 5.0]
models = {}
histories = {}

for beta in betas:
    print(f"\nTraining with β = {beta}...")
    model, history = train_vae(
        data, 
        latent_dim=2, 
        hidden_dim=64,
        n_epochs=50,
        batch_size=64,
        beta=beta
    )
    models[beta] = model
    histories[beta] = history

print("\nDone!")

In [ ]:
# Compare latent spaces
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for i, beta in enumerate(betas):
    model = models[beta]
    model.eval()
    
    with torch.no_grad():
        tensor_data = torch.FloatTensor(data).to(device)
        mu, log_var = model.encode(tensor_data)
        z = model.reparameterize(mu, log_var)
        x_recon, _, _ = model(tensor_data)
        
        mu = mu.cpu().numpy()
        x_recon = x_recon.cpu().numpy()
    
    # Latent space
    ax = axes[0, i]
    scatter = ax.scatter(mu[:, 0], mu[:, 1], c=labels, cmap='tab10', alpha=0.5, s=5)
    ax.set_xlabel('z₁')
    ax.set_ylabel('z₂')
    ax.set_title(f'β = {beta}\nLatent Space')
    ax.set_aspect('equal')
    
    # Reconstruction
    ax = axes[1, i]
    ax.scatter(x_recon[:500, 0], x_recon[:500, 1], c=labels[:500], cmap='tab10', alpha=0.5, s=5)
    ax.set_xlabel('x₁')
    ax.set_ylabel('x₂')
    
    # Compute final losses
    recon_final = histories[beta]['recon_loss'][-1]
    kl_final = histories[beta]['kl_loss'][-1]
    ax.set_title(f'Recon: {recon_final:.4f}\nKL: {kl_final:.4f}')
    ax.set_aspect('equal')

plt.suptitle('Effect of β on VAE: Information Bottleneck Trade-off', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n🔑 Observations:")
print("• β = 0: No KL → encoder memorizes, poor generation")
print("• β = 1: Standard VAE → balanced trade-off")
print("• β > 1: Strong regularization → smoother latent, worse reconstruction")
print("• High β: Risk of posterior collapse (KL → 0, ignores encoder)")

In [ ]:
# Rate-distortion trade-off
fig, ax = plt.subplots(figsize=(10, 6))

recons = [histories[beta]['recon_loss'][-1] for beta in betas]
kls = [histories[beta]['kl_loss'][-1] for beta in betas]

ax.scatter(kls, recons, s=200, c=betas, cmap='viridis', edgecolor='black', linewidth=2)

for beta, kl, recon in zip(betas, kls, recons):
    ax.annotate(f'β={beta}', (kl, recon), textcoords='offset points', 
                xytext=(10, 5), fontsize=12)

ax.set_xlabel('Rate (KL Divergence) [nats]')
ax.set_ylabel('Distortion (Reconstruction Loss)')
ax.set_title('Rate-Distortion Trade-off\nVAE as Information Bottleneck')
ax.grid(True, alpha=0.3)

# Add arrow showing trade-off direction
ax.annotate('', xy=(kls[-2], recons[-2]), xytext=(kls[1], recons[1]),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(np.mean(kls), np.mean(recons) + 0.002, 'Increasing β', color='red', fontsize=12, ha='center')

plt.tight_layout()
plt.show()

print("\n📊 Rate-Distortion Analysis:")
print("="*50)
print(f"{'β':<8} {'Rate (KL)':<15} {'Distortion (MSE)':<15}")
print("-"*50)
for beta, kl, recon in zip(betas, kls, recons):
    print(f"{beta:<8} {kl:<15.4f} {recon:<15.4f}")

## 8. Latent Space Interpolation

In [ ]:
def interpolate_latent(model: VAE, x1: np.ndarray, x2: np.ndarray, n_steps: int = 10):
    """Interpolate between two points in latent space."""
    
    model.eval()
    with torch.no_grad():
        # Encode both points
        x1_tensor = torch.FloatTensor(x1).unsqueeze(0).to(device)
        x2_tensor = torch.FloatTensor(x2).unsqueeze(0).to(device)
        
        mu1, _ = model.encode(x1_tensor)
        mu2, _ = model.encode(x2_tensor)
        
        # Interpolate in latent space
        alphas = np.linspace(0, 1, n_steps)
        interpolations = []
        latents = []
        
        for alpha in alphas:
            z = (1 - alpha) * mu1 + alpha * mu2
            x_interp = model.decode(z)
            interpolations.append(x_interp.cpu().numpy())
            latents.append(z.cpu().numpy())
        
        interpolations = np.array(interpolations).squeeze()
        latents = np.array(latents).squeeze()
    
    return interpolations, latents

# Select two points from different clusters
idx1 = np.where(labels == 0)[0][0]
idx2 = np.where(labels == 2)[0][0]

x1 = data[idx1]
x2 = data[idx2]

# Use the β=1 model
model = models[1.0]
interpolations, latents = interpolate_latent(model, x1, x2, n_steps=20)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Latent space interpolation
ax = axes[0]
model.eval()
with torch.no_grad():
    mu, _ = model.encode(torch.FloatTensor(data).to(device))
    mu = mu.cpu().numpy()

ax.scatter(mu[:, 0], mu[:, 1], c=labels, cmap='tab10', alpha=0.2, s=5)
ax.plot(latents[:, 0], latents[:, 1], 'r-', linewidth=2, marker='o', markersize=8)
ax.plot(latents[0, 0], latents[0, 1], 'gs', markersize=15, label='Start')
ax.plot(latents[-1, 0], latents[-1, 1], 'g^', markersize=15, label='End')
ax.set_xlabel('z₁')
ax.set_ylabel('z₂')
ax.set_title('Interpolation in Latent Space')
ax.legend()
ax.set_aspect('equal')

# Data space interpolation
ax = axes[1]
ax.scatter(data[:, 0], data[:, 1], c=labels, cmap='tab10', alpha=0.2, s=5)
ax.plot(interpolations[:, 0], interpolations[:, 1], 'r-', linewidth=2, marker='o', markersize=8)
ax.plot(x1[0], x1[1], 'gs', markersize=15, label='Start')
ax.plot(x2[0], x2[1], 'g^', markersize=15, label='End')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Corresponding Points in Data Space')
ax.legend()
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\n🔑 Observation:")
print("Interpolation in latent space produces smooth transitions in data space!")
print("This is because KL regularization ensures a continuous latent manifold.")

## Summary

### VAE Key Equations

| Component | Formula |
|-----------|--------|
| ELBO | $\mathcal{L} = \mathbb{E}_q[\log p(x|z)] - D_{KL}(q(z|x) \| p(z))$ |
| KL Loss | $\frac{1}{2}\sum_j(\mu_j^2 + \sigma_j^2 - 1 - \log\sigma_j^2)$ |
| Reparameterization | $z = \mu + \sigma \odot \epsilon$, $\epsilon \sim N(0,I)$ |
| β-VAE Loss | $\text{Recon} + \beta \cdot \text{KL}$ |

### Entropy/Information Perspective

1. **KL to prior** = Rate (information bottleneck)
2. **Reconstruction loss** = Distortion
3. **β** controls the rate-distortion trade-off
4. Higher β → more compression → smoother latent space
5. Lower β → less compression → better reconstruction

### Practical Insights

1. **Reparameterization** is essential for gradient flow
2. **KL regularization** enables generation from prior
3. **β tuning** is crucial for balancing quality vs smoothness
4. **Latent interpolation** reveals learned structure